<a href="https://colab.research.google.com/github/thazin31086/GitHub_NeturalNetworks/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import tensorflow as tf
import tensorflow_hub as hub
import keras.backend as K
import keras
import numpy as np
import pandas as pd
from sklearn import model_selection, preprocessing, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras import layers, models, optimizers

data = pd.read_csv('IssueDetailsSVF_20102019_Single.csv')
data = data.sample(frac=1)

y = list(data['Assignee']) 
x = list(data['Title']) 

le = preprocessing.LabelEncoder()
le.fit(y)

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

#80% / 20% train / test split:

x_enc = x
y_enc = encode(le, y)

train_size = int(len(x) * .8)

x_train = np.asarray(x_enc[:train_size])
y_train = np.asarray(y_enc[:train_size])

x_test = np.asarray(x_enc[train_size:])
y_test = np.asarray(y_enc[train_size:])


# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(data['Assignee'])
xtrain_tfidf =  tfidf_vect.transform(x_train)
xvalid_tfidf =  tfidf_vect.transform(x_test)

# Train Model
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    return metrics.accuracy_score(predictions, valid_y)

# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf, y_train, xvalid_tfidf)
print ("SVM, N-Gram Vectors: ", accuracy)

ValueError: ignored